In [422]:
import pandas as pd

In [423]:
#df = pd.concat(map(pd.read_csv, ["Subway_Data/turnstile_201205.txt", "Subway_Data/turnstile_201212.txt"]))
df = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_210102.txt")

In [424]:
#clean column names
df.rename(columns={'C/A': 'c_a', 'UNIT': 'unit', 'SCP': 'scp', 'STATION': 'station', 'LINENAME': 'linename', 'DIVISION': 'division', 'DATE': 'date','TIME': 'time', 'DESC': 'desc', 'ENTRIES': 'entries', df.columns[10]: 'exits'}, inplace=True)

#keep only subway divisions
df = df[(df['division'] == 'BMT') | (df['division'] == 'IND') | (df['division'] == 'IRT')]

#Add formatted timestamp column
df['timestamp'] = pd.to_datetime(df['date'] + " " + df['time'])

#add unique turnstile id
df['unique_turnstile_id'] = df['c_a'] + df['unit'] + ' ' + df['station'] + ' ' + df['scp']

#keep only needed columns
df = df.loc[:,['station', 'entries', 'exits', 'timestamp', 'unique_turnstile_id']]

#add day of week column
df['day_of_week'] = df.timestamp.dt.day_name()

df = df.reset_index()

In [425]:
#calculate actual volumes for each record (record minus previous record)
df['diff_entries'] = abs(df.entries.diff())
df['diff_exits'] = abs(df.exits.diff())
df['total_traffic'] = df.diff_entries + df.diff_exits

#record index in new column (needed to identify first instance of turnstile data)
df['record_index'] = df.index
df

,index,station,entries,exits,timestamp,unique_turnstile_id,day_of_week,diff_entries,diff_exits,total_traffic,record_index
0,0,59 ST,7508509,2557569,2020-12-26 03:00:00,A002R051 59 ST 02-00-00,Saturday,NaN,NaN,NaN,0
1,1,59 ST,7508511,2557581,2020-12-26 07:00:00,A002R051 59 ST 02-00-00,Saturday,2.0,12.0,14.0,1
2,2,59 ST,7508531,2557636,2020-12-26 11:00:00,A002R051 59 ST 02-00-00,Saturday,20.0,55.0,75.0,2
3,3,59 ST,7508610,2557667,2020-12-26 15:00:00,A002R051 59 ST 02-00-00,Saturday,79.0,31.0,110.0,3
4,4,59 ST,7508767,2557689,2020-12-26 19:00:00,A002R051 59 ST 02-00-00,Saturday,157.0,22.0,179.0,4
...,...,...,...,...,...,...,...,...,...,...,...
195464,208664,EASTCHSTER/DYRE,1559829041,1728236785,2021-01-01 04:00:00,R730R431 EASTCHSTER/DYRE 00-00-04,Friday,0.0,1.0,1.0,195464
195465,208665,EASTCHSTER/DYRE,1559829038,1728236790,2021-01-01 08:00:00,R730R431 EASTCHSTER/DYRE 00-00-04,Friday,3.0,5.0,8.0,195465
195466,208666,EASTCHSTER/DYRE,1559829033,1728236794,2021-01-01 12:00:00,R730R431 EASTCHSTER/DYRE 00-00-04,Friday,5.0,4.0,9.0,195466
195467,208667,EASTCHSTER/DYRE,1559829023,1728236798,2021-01-01 16:00:00,R730R431 EASTCHSTER/DYRE 00-00-04,Friday,10.0,4.0,14.0,195467


In [426]:
df.unique_turnstile_id.nunique()

4613

In [427]:
#identify first rows of unique turnstile data
df_first_rows = df.groupby('unique_turnstile_id').first()
df_first_rows

,index,station,entries,exits,timestamp,day_of_week,diff_entries,diff_exits,total_traffic,record_index
unique_turnstile_id,,,,,,,,,,
A002R051 59 ST 02-00-00,0,59 ST,7508509,2557569,2020-12-26 03:00:00,Saturday,2.000000e+00,1.200000e+01,1.400000e+01,0
A002R051 59 ST 02-00-01,43,59 ST,6657154,1508424,2020-12-26 03:00:00,Saturday,8.544930e+05,1.050441e+06,1.904934e+06,43
A002R051 59 ST 02-03-00,86,59 ST,1406769,5305160,2020-12-26 03:00:00,Saturday,5.252610e+06,3.795904e+06,9.048514e+06,86
A002R051 59 ST 02-03-01,129,59 ST,1529856,2379972,2020-12-26 03:00:00,Saturday,1.220860e+05,2.929410e+06,3.051496e+06,129
A002R051 59 ST 02-03-02,172,59 ST,6612655,8827961,2020-12-26 03:00:00,Saturday,5.080298e+06,6.443396e+06,1.152369e+07,172
...,...,...,...,...,...,...,...,...,...,...
R730R431 EASTCHSTER/DYRE 00-00-00,208459,EASTCHSTER/DYRE,1720336,3279225,2020-12-26 00:00:00,Saturday,2.062296e+09,1.646229e+08,2.226918e+09,195259
R730R431 EASTCHSTER/DYRE 00-00-01,208501,EASTCHSTER/DYRE,6894072,8998816,2020-12-26 00:00:00,Saturday,5.172865e+06,5.717972e+06,1.089084e+07,195301
R730R431 EASTCHSTER/DYRE 00-00-02,208543,EASTCHSTER/DYRE,3795776,3441977,2020-12-26 00:00:00,Saturday,3.100435e+06,5.560759e+06,8.661194e+06,195343


In [428]:
#df_first_rows.columns

In [429]:
#index of records of first turnstile rows
list_index = df_first_rows.record_index.values
len(list_index)

4613

In [430]:
#drop first rows from main df
df.drop(list_index, inplace=True)

In [431]:
#keep only data with traffic > 0
df = df[df['total_traffic'] > 0]

In [432]:
df = df.reset_index()


In [433]:
#drop record index column. no longer applies after re-indexing
df.drop('record_index', axis=1)

,level_0,index,station,entries,exits,timestamp,unique_turnstile_id,day_of_week,diff_entries,diff_exits,total_traffic
0,1,1,59 ST,7508511,2557581,2020-12-26 07:00:00,A002R051 59 ST 02-00-00,Saturday,2.0,12.0,14.0
1,2,2,59 ST,7508531,2557636,2020-12-26 11:00:00,A002R051 59 ST 02-00-00,Saturday,20.0,55.0,75.0
2,3,3,59 ST,7508610,2557667,2020-12-26 15:00:00,A002R051 59 ST 02-00-00,Saturday,79.0,31.0,110.0
3,4,4,59 ST,7508767,2557689,2020-12-26 19:00:00,A002R051 59 ST 02-00-00,Saturday,157.0,22.0,179.0
4,5,5,59 ST,7508852,2557704,2020-12-26 23:00:00,A002R051 59 ST 02-00-00,Saturday,85.0,15.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
162395,195464,208664,EASTCHSTER/DYRE,1559829041,1728236785,2021-01-01 04:00:00,R730R431 EASTCHSTER/DYRE 00-00-04,Friday,0.0,1.0,1.0
162396,195465,208665,EASTCHSTER/DYRE,1559829038,1728236790,2021-01-01 08:00:00,R730R431 EASTCHSTER/DYRE 00-00-04,Friday,3.0,5.0,8.0
162397,195466,208666,EASTCHSTER/DYRE,1559829033,1728236794,2021-01-01 12:00:00,R730R431 EASTCHSTER/DYRE 00-00-04,Friday,5.0,4.0,9.0
162398,195467,208667,EASTCHSTER/DYRE,1559829023,1728236798,2021-01-01 16:00:00,R730R431 EASTCHSTER/DYRE 00-00-04,Friday,10.0,4.0,14.0


In [434]:
df.sort_values(['total_traffic'],ascending=False).head(50)

,level_0,index,station,entries,exits,timestamp,unique_turnstile_id,day_of_week,diff_entries,diff_exits,total_traffic,record_index
135482,162621,175821,135 ST,1828739339,1728008021,2020-12-29 15:00:00,R307R207 135 ST 01-00-02,Tuesday,1.821543e+09,1.717955e+09,3.539498e+09,162621
135227,162242,175442,135 ST,855624591,922774443,2020-12-29 11:00:00,R306R207 135 ST 00-00-00,Tuesday,8.421683e+08,9.191326e+08,1.761301e+09,162242
132492,159135,172335,170 ST,50883726,50997243,2020-12-27 15:00:00,R284R243 170 ST 00-00-00,Sunday,5.088372e+07,5.099724e+07,1.018810e+08,159135
132491,159134,172334,170 ST,4,0,2020-12-27 12:00:00,R284R243 170 ST 00-00-00,Sunday,5.088368e+07,5.099719e+07,1.018809e+08,159134
10237,11579,11579,DELANCEY/ESSEX,18,27,2020-12-27 16:00:00,A061R142 DELANCEY/ESSEX 00-03-00,Sunday,3.039868e+06,2.113937e+06,5.153805e+06,11579
110683,132748,145948,79 ST,2238296,1358922,2020-12-30 20:00:00,R162R166 79 ST 00-00-00,Wednesday,2.198435e+06,1.313649e+06,3.512084e+06,132748
110690,132755,145955,79 ST,2238427,1359140,2020-12-31 12:00:00,R162R166 79 ST 00-00-00,Thursday,2.198351e+06,1.313637e+06,3.511988e+06,132755
110681,132746,145946,79 ST,2238056,1358832,2020-12-30 16:00:00,R162R166 79 ST 00-00-00,Wednesday,2.198309e+06,1.313669e+06,3.511978e+06,132746
110685,132750,145950,79 ST,2238359,1358948,2020-12-31 00:00:00,R162R166 79 ST 00-00-00,Thursday,2.198334e+06,1.313553e+06,3.511887e+06,132750
110679,132744,145944,79 ST,2237921,1358718,2020-12-30 12:00:00,R162R166 79 ST 00-00-00,Wednesday,2.198213e+06,1.313651e+06,3.511864e+06,132744


In [435]:
df.iloc[135480:135485,:]

,level_0,index,station,entries,exits,timestamp,unique_turnstile_id,day_of_week,diff_entries,diff_exits,total_traffic,record_index
135480,162619,175819,135 ST,7196336,10052494,2020-12-29 07:00:00,R307R207 135 ST 01-00-02,Tuesday,2.300000e+01,3.100000e+01,5.400000e+01,162619
135481,162620,175820,135 ST,7196454,10052705,2020-12-29 11:00:00,R307R207 135 ST 01-00-02,Tuesday,1.180000e+02,2.110000e+02,3.290000e+02,162620
135482,162621,175821,135 ST,1828739339,1728008021,2020-12-29 15:00:00,R307R207 135 ST 01-00-02,Tuesday,1.821543e+09,1.717955e+09,3.539498e+09,162621
135483,162622,175822,135 ST,1828739543,1728007791,2020-12-29 19:00:00,R307R207 135 ST 01-00-02,Tuesday,2.040000e+02,2.300000e+02,4.340000e+02,162622
135484,162623,175823,135 ST,1828739614,1728007631,2020-12-29 23:00:00,R307R207 135 ST 01-00-02,Tuesday,7.100000e+01,1.600000e+02,2.310000e+02,162623


In [436]:
df.iloc[107345:135485,:]

,level_0,index,station,entries,exits,timestamp,unique_turnstile_id,day_of_week,diff_entries,diff_exits,total_traffic,record_index
107345,128690,141890,TIMES SQ-42 ST,589355,192102,2020-12-31 07:00:00,R145R032 TIMES SQ-42 ST 00-06-01,Thursday,2.600000e+01,5.300000e+01,7.900000e+01,128690
107346,128691,141891,TIMES SQ-42 ST,589427,192480,2020-12-31 11:00:00,R145R032 TIMES SQ-42 ST 00-06-01,Thursday,7.200000e+01,3.780000e+02,4.500000e+02,128691
107347,128692,141892,TIMES SQ-42 ST,589542,193537,2020-12-31 15:00:00,R145R032 TIMES SQ-42 ST 00-06-01,Thursday,1.150000e+02,1.057000e+03,1.172000e+03,128692
107348,128693,141893,TIMES SQ-42 ST,589765,194936,2020-12-31 19:00:00,R145R032 TIMES SQ-42 ST 00-06-01,Thursday,2.230000e+02,1.399000e+03,1.622000e+03,128693
107349,128694,141894,TIMES SQ-42 ST,589845,195572,2020-12-31 23:00:00,R145R032 TIMES SQ-42 ST 00-06-01,Thursday,8.000000e+01,6.360000e+02,7.160000e+02,128694
...,...,...,...,...,...,...,...,...,...,...,...,...
135480,162619,175819,135 ST,7196336,10052494,2020-12-29 07:00:00,R307R207 135 ST 01-00-02,Tuesday,2.300000e+01,3.100000e+01,5.400000e+01,162619
135481,162620,175820,135 ST,7196454,10052705,2020-12-29 11:00:00,R307R207 135 ST 01-00-02,Tuesday,1.180000e+02,2.110000e+02,3.290000e+02,162620
135482,162621,175821,135 ST,1828739339,1728008021,2020-12-29 15:00:00,R307R207 135 ST 01-00-02,Tuesday,1.821543e+09,1.717955e+09,3.539498e+09,162621
135483,162622,175822,135 ST,1828739543,1728007791,2020-12-29 19:00:00,R307R207 135 ST 01-00-02,Tuesday,2.040000e+02,2.300000e+02,4.340000e+02,162622


In [437]:
# df_by_station = df.groupby(["station", "timestamp"]).total_traffic.max()
# len(df_by_station)

16300

In [438]:
df_by_station = df.groupby(["unique_turnstile_id", "timestamp"]).agg({'total_traffic': 'sum'}).reset_index()
df_by_station

,unique_turnstile_id,timestamp,total_traffic
0,A002R051 59 ST 02-00-00,2020-12-26 07:00:00,14.0
1,A002R051 59 ST 02-00-00,2020-12-26 11:00:00,75.0
2,A002R051 59 ST 02-00-00,2020-12-26 15:00:00,110.0
3,A002R051 59 ST 02-00-00,2020-12-26 19:00:00,179.0
4,A002R051 59 ST 02-00-00,2020-12-26 23:00:00,100.0
...,...,...,...
162381,R730R431 EASTCHSTER/DYRE 00-00-04,2021-01-01 04:00:00,1.0
162382,R730R431 EASTCHSTER/DYRE 00-00-04,2021-01-01 08:00:00,8.0
162383,R730R431 EASTCHSTER/DYRE 00-00-04,2021-01-01 12:00:00,9.0
162384,R730R431 EASTCHSTER/DYRE 00-00-04,2021-01-01 16:00:00,14.0


In [439]:
df_by_station.sort_values(by="total_traffic", ascending=False)

,unique_turnstile_id,timestamp,total_traffic
135468,R307R207 135 ST 01-00-02,2020-12-29 15:00:00,3.539498e+09
135213,R306R207 135 ST 00-00-00,2020-12-29 11:00:00,1.761301e+09
132478,R284R243 170 ST 00-00-00,2020-12-27 15:00:00,1.018810e+08
132477,R284R243 170 ST 00-00-00,2020-12-27 12:00:00,1.018809e+08
110679,R162R166 79 ST 00-00-00,2020-12-30 20:00:00,7.023882e+06
...,...,...,...
51199,N083R138 W 4 ST-WASH SQ 01-03-00,2020-12-27 04:00:00,1.000000e+00
8440,A050R088 CORTLANDT ST 00-03-00,2020-12-28 04:00:00,1.000000e+00
89858,N531R129 BERGEN ST 01-06-01,2021-01-01 12:00:00,1.000000e+00
108645,R155R116 50 ST 01-00-06,2021-01-01 07:00:00,1.000000e+00
